<a href="https://colab.research.google.com/github/dr-kinder/playground/blob/master/gmsh_stencil_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import

In [ ]:
try:
    # Import gmsh library for generating meshes.
    import gmsh
except ImportError:
    # If it is not available, install it.  Then import it.
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

In [ ]:
try:
    # Import FEniCSx libraries for finite element analysis.
    import dolfinx
except ImportError:
    # If they are not found, install them.  Then import them.
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [ ]:
try:
    # Import multiphenicsx, mainly for plotting.
    import multiphenicsx
except ImportError:
    # If they are not found, install them.
    !pip3 install "multiphenicsx@git+https://github.com/multiphenics/multiphenicsx.git@8b97b4e"
    import multiphenicsx

In [ ]:
# Everything should be installed now.
# Import the rest of what we need.

import dolfinx.fem
import dolfinx.io
import gmsh
import mpi4py.MPI
import numpy as np
import petsc4py.PETSc
import ufl
import multiphenicsx.io

# Build a model with GMSH

This model has several separate parts that we want to combine into a single mesh.  GMSH won't complain if surfaces overlap, but FEniCSx will.
There are several ways to define a single mesh with multiple entities using GMSH.  (See [the GMSH tutorials](https://gitlab.onelab.info/gmsh/gmsh/-/tree/gmsh_4_9_5/tutorials/python) for some examples — in particular, `t4.py` and `t5.py`.)

This example creates a stencil.  We define the "universe" to be the entire region of interest — a large circular disk.  The "universe" is actually three distinct regions in this system: two magnets and the background.  We define the shape of the magnets first.  Then we define the background to be the universe, but with holes cut out for the magnets.  And then we define surfaces for the two magnets.  We then introduce the physical objects: the boundaries, the background, and the two magnets.  They must fit together like pieces of a jigsaw puzzle.

Plotting the mesh shows the three regions.  They are tagged in the model, too, so we can assign different properties to each.

In [2]:
# Define the center of the circle.
x0 = 0
y0 = 0
z0 = 0

#Define the top magnet
P1X = -2.5
P1Y = 4.33
P2X = -2.5
P2Y = 2.17
P3X = 2.5
P3Y = 2.17
P4X = 2.5
P4Y = 4.33

#Define Bottom Magnet
B1X = -2.5
B1Y = -4.33
B2X = -2.5
B2Y = -1.76
B3X = -2
B3Y = -1.76
B4X = -2
B4Y = -2.76
B5X = 2
B5Y = -2.76
B6X = 2
B6Y = -1.76
B7X = 2.5
B7Y = -1.76
B8X = 2.5
B8Y = -4.33

# Define the radius of the circle.
r0 = 10

# Tell the modeling program how many dimensions we are using.
dim = 2

# Grid size parameter.  Make it smaller for higher resolution.
delta = 0.2

In [ ]:
# Let's check the shape before trying to get GMSH to make a model ...
import matplotlib.pyplot as plt

x1 = [B1X, B2X, B3X, B4X, B5X, B6X, B7X, B8X, B1X]
y1 = [B1Y, B2Y, B3Y, B4Y, B5Y, B6Y, B7Y, B8Y, B1Y]

x2 = [P1X, P2X, x0, P3X, P4X, P1X]
y2 = [P1Y, P2Y, y0, P3Y, P4Y, P1Y]

plt.plot(x1,y1, 'b-o')
plt.plot(x2,y2,'r-o')

plt.axis("equal")

In [ ]:
# Create the model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define points: center of circle and two points on opposite sides.
p0 = gmsh.model.geo.addPoint(x0,y0, z0, delta)
p1 = gmsh.model.geo.addPoint(x0, y0-r0, z0, delta)
p2 = gmsh.model.geo.addPoint(x0, y0+r0,z0, delta)

#Define points: Top Magnet
T1 = gmsh.model.geo.addPoint(P1X, P1Y, z0, delta)
T2 = gmsh.model.geo.addPoint(P2X, P2Y, z0, delta)
T3 = gmsh.model.geo.addPoint(x0,y0, z0, delta)
T4 = gmsh.model.geo.addPoint(P3X, P3Y, z0, delta)
T5 = gmsh.model.geo.addPoint(P4X, P4Y, z0, delta)

#Define points: Bottom Magnet
B1 = gmsh.model.geo.addPoint(B1X, B1Y, z0, delta)
B2 = gmsh.model.geo.addPoint(B2X, B2Y, z0, delta)
B3 = gmsh.model.geo.addPoint(B3X, B3Y, z0, delta)
B4 = gmsh.model.geo.addPoint(B4X, B4Y, z0, delta)
B5 = gmsh.model.geo.addPoint(B5X, B5Y, z0, delta)
B6 = gmsh.model.geo.addPoint(B6X, B6Y, z0, delta)
B7 = gmsh.model.geo.addPoint(B7X, B7Y, z0, delta)
B8 = gmsh.model.geo.addPoint(B8X, B8Y, z0, delta)

#Define: Top magnet shape
L1 = gmsh.model.geo.addLine(T1, T2)
L2 = gmsh.model.geo.addLine(T2, T3)
L3 = gmsh.model.geo.addLine(T3, T4)
L4 = gmsh.model.geo.addLine(T4, T5)
L5 = gmsh.model.geo.addLine(T5, T1)
topLoop = gmsh.model.geo.addCurveLoop([L1, L2, L3, L4,L5])

#Define: Bottom magnet shape
D1 = gmsh.model.geo.addLine(B1, B2)
D2 = gmsh.model.geo.addLine(B2, B3)
D3 = gmsh.model.geo.addLine(B3, B4)
D4 = gmsh.model.geo.addLine(B4, B5)
D5 = gmsh.model.geo.addLine(B5, B6)
D6 = gmsh.model.geo.addLine(B6, B7)
D7 = gmsh.model.geo.addLine(B7, B8)
D8 = gmsh.model.geo.addLine(B8, B1)
bottomLoop = gmsh.model.geo.addCurveLoop([D1, D2, D3, D4, D5, D6, D7, D8])

# Define two semicircular arcs that will be joined into a circle.
# The circular disk is the "universe" --- the entire region of interest.
c0 = gmsh.model.geo.addCircleArc(p1, p0, p2)
c1 = gmsh.model.geo.addCircleArc(p2, p0, p1)
universe = gmsh.model.geo.addCurveLoop([c0,c1])

# The "background" is the universe, minus holes for the objects inside.
background = gmsh.model.geo.addPlaneSurface([loop,topLoop,bottomLoop])

# The two objects fill in the holes in the background.
topMagnet = gmsh.model.geo.addPlaneSurface([topLoop])
bottomMagnet = gmsh.model.geo.addPlaneSurface([bottomLoop])

# Update the model with all of the features we add.
gmsh.model.geo.synchronize()

# Identify the physical objects: boundaries and surfaces.
gmsh.model.addPhysicalGroup(1, [c0,c1], 1)
gmsh.model.addPhysicalGroup(2, [background], 1)
gmsh.model.addPhysicalGroup(1, [L1, L2, L3, L4,L5], 2)
gmsh.model.addPhysicalGroup(2, [topMagnet], 2)
gmsh.model.addPhysicalGroup(1, [D1, D2, D3, D4, D5, D6, D7, D8], 3)
gmsh.model.addPhysicalGroup(2, [bottomMagnet], 3)

# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

# Close the mesh generating program.
gmsh.finalize()

In [ ]:
# Plot the entire mesh.
multiphenicsx.io.plot_mesh(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
# Plot the subdomains that FEniCSx has identified.
multiphenicsx.io.plot_mesh_tags(subdomains)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
# Inspect the boundaries of the elements and the system.
multiphenicsx.io.plot_mesh_tags(boundaries)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…